In [1]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.optimizers import SGD
import numpy as np
import os 
from PIL import Image

In [2]:
def get_jpg_path(file_dir='datasets\\birds\\train\\'):
    path=[]
    clazz=[]
    for root, dirs, files in os.walk(file_dir): 
        if len(dirs)==0:
            cls=root[-3:]
            cls=[int(cls)]*len(files)
            ph=[root+"\\"+p for p in files]
            path.extend(ph)
            clazz.extend(cls)
    return path,clazz
path,clazz=get_jpg_path('datasets\\birds\\train\\')

In [3]:
def onehot_label(clazz):
    index=0
    is_insight=set()
    hot=[0]*90
    onehot=[]
    for i in clazz:
        if i not in is_insight:
            is_insight.add(i)
            hot=[0]*90
            hot[index]=1
            index+=1
        onehot.append(hot)
    onehot=np.asarray(onehot)
    return onehot

In [4]:
onehot=onehot_label(clazz)
onehot.shape

(3830, 90)

In [5]:
def reszie_jpg(path):
    imgs=np.zeros(shape=(3830,224,224,3))
    index=0
    for p in path:
        im=Image.open(p)
        im=im.resize((224,224))
        npim=np.asarray(im)
        if npim.shape!=(224,224,3):
            print(npim.shape,index)
        else:
            imgs[index]=npim
        index+=1
    return imgs

In [6]:
imgs=reszie_jpg(path)
imgs.shape

(224, 224) 84
(224, 224) 375
(224, 224) 1358
(224, 224) 1800
(224, 224) 1942
(224, 224) 2218


(3830, 224, 224, 3)

In [7]:
model = Sequential()
# BLOCK 1
model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block1_conv1', input_shape = (224, 224, 3)))   
model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block1_conv2'))
model.add(MaxPooling2D(pool_size = (4, 4), strides = (4, 4), name = 'block1_pool'))
 
# BLOCK2
model.add(Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block2_conv1'))   
model.add(Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block2_conv2'))
model.add(MaxPooling2D(pool_size = (4, 4), strides = (4, 4), name = 'block2_pool'))
 
# BLOCK3
model.add(Conv2D(filters = 256, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block3_conv1'))   
model.add(Conv2D(filters = 256, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block3_conv2'))
model.add(Conv2D(filters = 256, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block3_conv3'))
model.add(MaxPooling2D(pool_size = (4, 4), strides = (4, 4), name = 'block3_pool'))
 
# # BLOCK4
# model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block4_conv1'))   
# model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block4_conv2'))
# model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block4_conv3'))
# model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), name = 'block4_pool'))
 
# # BLOCK5
# model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block5_conv1'))   
# model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block5_conv2'))
# model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block5_conv3'))
# model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), name = 'block5_pool'))
 
model.add(Flatten())
model.add(Dense(1024, activation = 'relu', name = 'fc1'))
model.add(Dropout(rate=0.5))
model.add(Dense(1024, activation = 'relu', name = 'fc2'))
model.add(Dropout(rate=0.5))
model.add(Dense(90, activation = 'softmax', name = 'prediction'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 56, 56, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 56, 56, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 56, 56, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 14, 14, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 14, 14, 256)       295168    
__________

In [9]:
model.compile(optimizer=SGD(0.1),
              #loss='mse',
              loss='categorical_crossentropy',
              #loss='binary_crossentropy',
              metrics=['accuracy'])

In [11]:
from sklearn.model_selection import train_test_split
xtr,xte,ytr,yte=train_test_split(imgs,onehot,test_size=0.3,random_state=1)

In [12]:
model.fit(xtr, ytr, epochs=1)

Instructions for updating:
Use tf.cast instead.
2681/2681 [==============================] - 17s 6ms/sample - loss: 15.9496 - acc: 0.0086


In [34]:
rs=model.predict(imgs[0:10])
for i in range(len(rs)):
    print(rs[i])
    print(onehot[i])

[0.01072291 0.01073736 0.01125581 0.01095123 0.0113096  0.0107666
 0.01106321 0.01082318 0.01129509 0.01089017 0.01120081 0.0111504
 0.01118319 0.01125955 0.01096459 0.0114021  0.01105785 0.01115866
 0.01116157 0.01091313 0.01103678 0.01116761 0.01089413 0.01094126
 0.01089955 0.01081029 0.01145709 0.01129242 0.01128486 0.01131305
 0.01096622 0.01094435 0.01094804 0.01117639 0.01106422 0.01119096
 0.01124079 0.01117383 0.01109465 0.01147254 0.01104987 0.01121317
 0.01110896 0.01131056 0.0114742  0.01111839 0.01111995 0.01107155
 0.01121696 0.01123413 0.01079423 0.01094814 0.01092383 0.01124684
 0.01107778 0.01112877 0.01088976 0.01136811 0.0112252  0.01028229
 0.01106653 0.01084825 0.01123654 0.01096233 0.01123914 0.01120788
 0.01081444 0.01118283 0.01129275 0.01107362 0.01164506 0.01120591
 0.01084604 0.01109967 0.01090941 0.01093801 0.01128644 0.01105327
 0.01169732 0.01086901 0.01095048 0.01121553 0.01123411 0.01128309
 0.01134731 0.0112013  0.01100412 0.01116736 0.01137801 0.011305